Start of the Neural Network

In [9]:
import pandas as pd
import tensorflow as tf
import matplotlib as mpl


AttributeError: Module 'numpy.core' has no attribute 'numerictypes'

In [61]:
from tensorflow import keras

In [41]:
dfs = []
for year in range(2000, 2011):
    dfs.append(pd.read_csv(f'{year}.csv'))
df = pd.concat(dfs)
len(df)
df = df.drop('id', axis = 1)

In [42]:
print(df.head())



   popularity  danceability  energy  key  loudness  mode  speechiness  \
0          86         0.371   0.268    1   -10.506     1       0.0281   
1          90         0.429   0.661   11    -7.227     1       0.0281   
2          87         0.949   0.661    5    -4.244     0       0.0572   
3          81         0.545   0.865   11    -5.708     0       0.0286   
4          82         0.843   0.806    4    -5.946     0       0.2690   

   acousticness  instrumentalness  liveness  valence    tempo  duration_ms  \
0       0.74800          0.051700    0.1040    0.165  102.617       227093   
1       0.00239          0.000121    0.2340    0.285  173.372       266773   
2       0.03020          0.000000    0.0454    0.760  104.504       284200   
3       0.00664          0.000011    0.1680    0.543   99.009       233933   
4       0.14300          0.000000    0.0771    0.613   94.948       270507   

   time_signature  
0               3  
1               4  
2               4  
3           

What we gotta predict = Popularity    
What we have = [danceability  energy  key  loudness  mode  speechiness acousticness  instrumentalness  liveness  valence  tempo  duration_ms time_signature]

In [56]:
#Variable Selection
import sklearn
from sklearn.ensemble import RandomForestRegressor

def select_features_for_popularity(df):
    # Define the target variable (popularity) and features
    target = df['popularity']
    features = [
        'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'time_signature'
    ]

    # Create a DataFrame with only the selected features
    selected_features_df = df[features]

    # Train a Random Forest model to estimate feature importance
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(selected_features_df, target)

    # Get feature importances
    feature_importances = (pd.Series(rf.feature_importances_, index=features))
    feature_importances = feature_importances.sort_values(ascending=False)

    # Sort features by importance in descending order
    selected_features = feature_importances.sort_values(ascending=False).index.tolist()

    return selected_features, feature_importances

# Example usage:
# Assuming you have a DataFrame 'df' with 'popularity' and other features
# selected_features = select_features_for_popularity(df)
# print("Selected Features for Predicting Popularity:")
# print(selected_features)


In [58]:
ouput = select_features_for_popularity(df)

(['loudness',  
  'duration_ms',   
  'tempo',   
  'danceability',   
  'speechiness',    
  'liveness',   
  'acousticness',   
  'valence',   
  'energy',   
  'instrumentalness',    
  'key',    
  'mode',    
  'time_signature'],    
 loudness            0.111960    
 duration_ms         0.100432   
 tempo               0.100021    
 danceability        0.099350    
 speechiness         0.097961    
 liveness            0.096390    
 acousticness        0.092973    
 valence             0.092488    
 energy              0.086707    
 instrumentalness    0.057583    
 key                 0.047823    
 mode                0.010578    
 time_signature      0.005734    
 dtype: float64)    

In [ ]:
# List of columns to keep
columns_to_keep = ['popularity', 'loudness', 'duration_ms', 'tempo', 'danceability', 'speechiness', 'liveness', 'acousticness', 'valence', 'energy']

# Create a new DataFrame with only the selected columns
df_filtered = df[columns_to_keep]

print(df_filtered.head())

   popularity  loudness  duration_ms    tempo  danceability  speechiness  \
0          86   -10.506       227093  102.617         0.371       0.0281   
1          90    -7.227       266773  173.372         0.429       0.0281   
2          87    -4.244       284200  104.504         0.949       0.0572   
3          81    -5.708       233933   99.009         0.545       0.0286   
4          82    -5.946       270507   94.948         0.843       0.2690   

   liveness  acousticness  valence  energy  
0    0.1040       0.74800    0.165   0.268  
1    0.2340       0.00239    0.285   0.661  
2    0.0454       0.03020    0.760   0.661  
3    0.1680       0.00664    0.543   0.865  
4    0.0771       0.14300    0.613   0.806  


In [8]:

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM, Dense


# Load your DataFrame (replace 'your_data.csv' with your actual data file)

# Define input features and target variable
input_features = ['loudness', 'duration_ms', 'tempo', 'danceability', 'speechiness', 'liveness', 'acousticness', 'valence', 'energy']
target_variable = 'popularity'

# Split the data into training and testing sets
X = df_filtered[input_features].values
y = df_filtered[target_variable].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize (normalize) the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build a simple neural network model
model = keras.Sequential([
    layers.Input(shape=(len(input_features),)),
    layers.LSTM(128),
    layers.Dense(32),
    #layers.Dense(16),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=2)

# Evaluate the model on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Mean Absolute Error: {test_mae}")

# Make predictions
predictions = model.predict(X_test)

# You can use predictions for further analysis or visualization


AttributeError: Module 'numpy.core' has no attribute 'numerictypes'

In [ ]:
print(predictions)

[[58.215065]
 [58.722282]
 [55.942528]
 ...
 [55.668953]
 [56.090195]
 [56.9917  ]]


In [ ]:
while True:
    try:
        # Input data from the user
        loudness = -10.506
        duration_ms = 227093
        tempo = 102.617
        danceability = 0.371
        speechiness = 0.0281
        liveness = 0.104
        acousticness = 0.748
        valence = 0.165
        energy = 0.268

        # Create a feature vector from user inputs
        user_input = scaler.transform([[loudness, duration_ms, tempo, danceability, speechiness, liveness, acousticness, valence, energy]])

        # Predict the popularity using the model
        prediction = model.predict(user_input)

        # Display the predicted popularity
        print(f"Predicted Popularity: {prediction[0][0]}")

    except ValueError:
        print("Invalid input. Please enter numeric values for all features.")

    # Ask if the user wants to continue
    again = input("Do you want to predict another popularity value? (yes/no): ").strip().lower()
    if again != "yes":
        break

NameError: name 'scaler' is not defined

'liveness', 'acousticness', 'valence', 'energy